# 1.) Load libraries/dataset

In [1]:
# Load libraries
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import socket

from itertools import product
from os.path import isfile

from IPython.display import display_markdown

sns.set(style='ticks', color_codes=True)

# Load datasets
data_path_flows = './datasets/assignment10/TrafficLabelling_/'
data_path_ML = './datasets/assignment10/MachineLearningCVE/'

data_paths = {}
data_paths['Dataset 1'] = 'Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv'
data_paths['Dataset 2'] = 'Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv'
data_paths['Dataset 3'] = 'Friday-WorkingHours-Morning.pcap_ISCX.csv'
data_paths['Dataset 4'] = 'Monday-WorkingHours.pcap_ISCX.csv'
data_paths['Dataset 5'] = 'Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv'
data_paths['Dataset 6'] = 'Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv'
data_paths['Dataset 7'] = 'Tuesday-WorkingHours.pcap_ISCX.csv'
data_paths['Dataset 8'] = 'Wednesday-workingHours.pcap_ISCX.csv'

# Functionalized in case I want to look at the other set of datasets
def load_datasets(parent_path=data_path_flows):
    datasets = data_paths.copy()
    for path in data_paths:
        datasets[path] = parent_path + data_paths[path]
        
    # Safety-Check path and Open/Convert to dataframes
    # Bulk loading probably isn't best practice.
    for dataset in datasets:
        if not isfile(datasets[dataset]):
            print(f"Dataset {dataset} not found. Please check that the dataset exists and the path is correct.")

        else:
            try: 
                datasets[dataset] = pd.read_csv(datasets[dataset], encoding='utf-8')
            except:
                datasets[dataset] = pd.read_csv(datasets[dataset], encoding='cp1252')
                
    # Trim Extra Whitespace from Column Names
    # I don't understand why pandas doesn't do this automagically
    for dataset in datasets:
        df = datasets[dataset]
        df.rename(columns=lambda col: col.strip(), inplace=True)

    return datasets
    
# Load the default dataset
datasets = load_datasets()

C:\Users\whm00\miniconda3\envs\AML\lib\site-packages\IPython\core\interactiveshell.py:3337: DtypeWarning: Columns (0,1,3,6,84) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [2]:
# Drop a few columns
# The one's ID'd in the above warning are useless to us (except 84 which is the label)
drop = ['Flow ID', 'Source IP', 'Destination IP', 'Timestamp']

for dataset in datasets:
    df = datasets[dataset]
    df.drop(columns=drop, inplace=True)
    
# This should leave us with a dataset that looks like the ML dataset with Protocol and Source Port added  
def check_congruity():
    datasets2 = load_datasets(data_path_ML)

    for dataset in datasets:
        df1 = datasets[dataset]
        df2 = datasets2[dataset]
    
        print(df1.shape, df2.shape)
        assert(df1.shape[0] == df2.shape[0])
        assert(df1.shape[1] != df2.shape[1])

#check_congruity() # For Sanity check

In [3]:
# Show our datasets
for dataset in datasets:
    display_markdown('### '+ dataset, raw=True)
    display(datasets[dataset].head())

### Dataset 1

,Source Port,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,443,54865,6,3,2,0,12,0,6,6,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
1,80,55054,6,109,1,1,6,6,6,6,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
2,80,55055,6,52,1,1,6,6,6,6,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
3,443,46236,6,34,1,1,6,6,6,6,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN
4,443,54863,6,3,2,0,12,0,6,6,...,20,0.0,0.0,0,0,0.0,0.0,0,0,BENIGN


### Dataset 2

,Source Port,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,35396,22,6,1266342,41,44,2664,6954,456,0,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
1,60058,22,6,1319353,41,44,2664,6954,456,0,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2,35396,22,6,160,1,1,0,0,0,0,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
3,35398,22,6,1303488,41,42,2728,6634,456,0,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
4,22,35396,6,77,1,2,0,0,0,0,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN


### Dataset 3

...

,Source Port,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,33898.0,389.0,6.0,113095465.0,48.0,24.0,9668.0,10012.0,403.0,0.0,...,32.0,203985.500,5.758373e+05,1629110.0,379.0,13800000.0,4.277541e+06,16500000.0,6737603.0,BENIGN
1,33904.0,389.0,6.0,113473706.0,68.0,40.0,11364.0,12718.0,403.0,0.0,...,32.0,178326.875,5.034269e+05,1424245.0,325.0,13800000.0,4.229413e+06,16500000.0,6945512.0,BENIGN
2,0.0,0.0,0.0,119945515.0,150.0,0.0,0.0,0.0,0.0,0.0,...,0.0,6909777.333,1.170000e+07,20400000.0,6.0,24400000.0,2.430000e+07,60100000.0,5702188.0,BENIGN
3,59135.0,443.0,6.0,60261928.0,9.0,7.0,2330.0,4221.0,1093.0,0.0,...,20.0,0.000,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0,0.0,BENIGN
4,59555.0,53.0,17.0,269.0,2.0,2.0,102.0,322.0,51.0,51.0,...,32.0,0.000,0.000000e+00,0.0,0.0,0.0,0.000000e+00,0.0,0.0,BENIGN


### Dataset 7

,Source Port,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,49182,88,6,640,7,4,440,358.0,220,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
1,49183,88,6,900,9,4,600,2944.0,300,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2,49184,88,6,1205,7,4,2776,2830.0,1388,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
3,49185,88,6,511,7,4,452,370.0,226,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
4,49186,88,6,773,9,4,612,2944.0,306,0,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN


### Dataset 8

,Source Port,Destination Port,Protocol,Flow Duration,Total Fwd Packets,Total Backward Packets,Total Length of Fwd Packets,Total Length of Bwd Packets,Fwd Packet Length Max,Fwd Packet Length Min,...,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label
0,49459,80,6,38308,1,1,6,6.0,6,6,...,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
1,49453,389,6,479,11,5,172,326.0,79,0,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
2,46124,88,6,1095,10,6,3150,3150.0,1575,0,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
3,49454,389,6,15206,17,12,3452,6660.0,1313,0,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN
4,46126,88,6,1092,9,6,3150,3152.0,1575,0,...,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,BENIGN


In [4]:
# Create an abriged protocol lookup table
# https://stackoverflow.com/questions/37004965/how-to-turn-protocol-number-to-name-with-python
protocols = {num:name[8:] for name,num in vars(socket).items() if name.startswith("IPPROTO")}

print(protocols)

{0: 'HOPOPTS', 1: 'ICMP', 2: 'IGMP', 3: 'GGP', 4: 'IPV4', 41: 'IPV6', 6: 'TCP', 8: 'EGP', 12: 'PUP', 17: 'UDP', 22: 'IDP', 77: 'ND', 43: 'ROUTING', 44: 'FRAGMENT', 50: 'ESP', 51: 'AH', 58: 'ICMPV6', 59: 'NONE', 60: 'DSTOPTS', 103: 'PIM', 132: 'SCTP', 255: 'RAW', 256: 'MAX', 78: 'ICLFXBM', 5: 'ST', 7: 'CBT', 9: 'IGP', 27: 'RDP', 113: 'PGM', 115: 'L2TP'}


# 2). Explore a little

In [5]:
active_df = datasets['Dataset 1']

In [6]:
# Check for nulls
def check_for_nulls(dataset):
    df = datasets[dataset]
    
    display_markdown('### '+ dataset + ' nulls', raw=True)
    display(df.loc[:, df.isnull().any()].columns)
    
check_for_nulls('Dataset 1')

### Dataset 1 nulls

Index(['Flow Bytes/s'], dtype='object')

In [7]:
# How many nulls are there?
def check_null_count(dataset):
    df = datasets[dataset]
    
    num_nulls = df[df.isnull().any(axis=1)].shape[0]
    display_markdown(f"**{dataset}** has {num_nulls} rows with nulls out of {df.shape[0]} rows. " +
                      f"({num_nulls / df.shape[0]:.5f}%)", raw=True)
    
check_null_count('Dataset 1')

**Dataset 1** has 4 rows with nulls out of 225745 rows. (0.00002%)

So we only have to worry about imputing on one column and that column is well represented in the data.

In [8]:
# Check our datatypes
def check_dtypes(dataset):
    display_markdown('### '+ dataset + ' dtypes', raw=True)
    display(datasets[dataset].dtypes.value_counts())
    
check_dtypes('Dataset 1')

### Dataset 1 dtypes

int64      56
float64    24
object      1
dtype: int64

And the only 'weird' datatype is the label (a string).

In [9]:
# Check label classes
# (This is where I discovered excess whitespace *sad Will noises*)
def check_feat_classes(dataset, feat):
    df = datasets[dataset]
    
    display(df[feat].unique())
    
check_feat_classes('Dataset 1', 'Label')

array(['BENIGN', 'DDoS'], dtype=object)

In [10]:
# Let's see all them feature names real quick.
def print_feature_names(dataset, per_line=4, datasets=datasets):
    df = datasets[dataset]
    outp = ''
    
    for i in range(0, df.shape[1], per_line):
        max_len = i+per_line if i+per_line < df.shape[1] else df.shape[1]
        
        for j in range(i, max_len):
            temp = '\"' + df.columns[j] + '\"'
            temp = f'{temp:30s}'
            
            outp += temp
            
        outp += '\n'
        
    outp = outp[:len(outp)-1]
    
    print(outp)
    
print_feature_names('Dataset 1')

"Source Port"                 "Destination Port"            "Protocol"                    "Flow Duration"               
"Total Fwd Packets"           "Total Backward Packets"      "Total Length of Fwd Packets" "Total Length of Bwd Packets" 
"Fwd Packet Length Max"       "Fwd Packet Length Min"       "Fwd Packet Length Mean"      "Fwd Packet Length Std"       
"Bwd Packet Length Max"       "Bwd Packet Length Min"       "Bwd Packet Length Mean"      "Bwd Packet Length Std"       
"Flow Bytes/s"                "Flow Packets/s"              "Flow IAT Mean"               "Flow IAT Std"                
"Flow IAT Max"                "Flow IAT Min"                "Fwd IAT Total"               "Fwd IAT Mean"                
"Fwd IAT Std"                 "Fwd IAT Max"                 "Fwd IAT Min"                 "Bwd IAT Total"               
"Bwd IAT Mean"                "Bwd IAT Std"                 "Bwd IAT Max"                 "Bwd IAT Min"                 
"Fwd PSH Flags"               "B

In [12]:
# Plot em
active_df.plot.hist()

ValueError: autodetected range of [-12000000.0, inf] is not finite